# Data Organization: Matrix Structure

>**Reference**: Hsiang-Fu Yu, Nikhil Rao, Inderjit S. Dhillon, 2016. [*Temporal regularized matrix factorization for high-dimensional time series prediction*](http://www.cs.utexas.edu/~rofuyu/papers/tr-mf-nips.pdf). 30th Conference on Neural Information Processing Systems (*NIPS 2016*), Barcelona, Spain.

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{f},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We express spatio-temporal dataset as a matrix $Y\in\mathbb{R}^{m\times f}$ with $m$ rows (e.g., locations) and $f$ columns (e.g., discrete time intervals),

$$Y=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{m1} & y_{m2} & \cdots & y_{mf} \\ \end{array} \right]\in\mathbb{R}^{m\times f}.$$

# Temporal Regularized Matrix Factorization(TRMF)
Temporal Regularized Matrix Factorization (TRMF) framework is an approach to incorporate temporal dependencies into matrix factorization models which use well-studied time series models to describe temporal dependencies
among ${\boldsymbol{x}_t}$ explicitly.Such models take the form:

$$\boldsymbol{x}_{t}\approx\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}$$

where this autoregressive (AR) is specialized by a lag set $\mathcal{L}=\left\{l_1,l_2,...,l_d\right\}$ (e.g., $\mathcal{L}=\left\{1,2,144\right\}$) and weights $\boldsymbol{\theta}_{l}\in\mathbb{R}^{r},\forall l$, and we further define

$$\mathcal{R}_{AR}\left(X\mid \mathcal{L},\Theta,\eta\right)=\frac{1}{2}\sum_{t=l_d+1}^{f}\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)^T\left(\boldsymbol{x}_{t}-\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}\circledast\boldsymbol{x}_{t-l}\right)+\frac{\eta}{2}\sum_{t=1}^{f}\boldsymbol{x}_{t}^T\boldsymbol{x}_{t}.$$

Thus, TRMF-AR is given by solving

$$\min_{W,X,\Theta}\frac{1}{2}\underbrace{\sum_{(i,t)\in\Omega}\left(y_{it}-\boldsymbol{w}_{i}^T\boldsymbol{x}_{t}\right)^2}_{\text{sum of squared residual errors}}+\lambda_{w}\underbrace{\mathcal{R}_{w}\left(W\right)}_{W-\text{regularizer}}+\lambda_{x}\underbrace{\mathcal{R}_{AR}\left(X\mid \mathcal{L},\Theta,\eta\right)}_{\text{AR-regularizer}}+\lambda_{\theta}\underbrace{\mathcal{R}_{\theta}\left(\Theta\right)}_{\Theta-\text{regularizer}}$$

where $\mathcal{R}_{w}\left(W\right)=\frac{1}{2}\sum_{i=1}^{m}\boldsymbol{w}_{i}^T\boldsymbol{w}_{i}$ and $\mathcal{R}_{\theta}\left(\Theta\right)=\frac{1}{2}\sum_{l\in\mathcal{L}}\boldsymbol{\theta}_{l}^T\boldsymbol{\theta}_{l}$ are regularization terms.

# Matrix Computation Concepts

## Kronecker product

- **Definition**:

Given two matrices $A\in\mathbb{R}^{m_1\times n_1}$ and $B\in\mathbb{R}^{m_2\times n_2}$, then, the **Kronecker product** between these two matrices is defined as

$$A\otimes B=\left[ \begin{array}{cccc} a_{11}B & a_{12}B & \cdots & a_{1m_2}B \\ a_{21}B & a_{22}B & \cdots & a_{2m_2}B \\ \vdots & \vdots & \ddots & \vdots \\ a_{m_11}B & a_{m_12}B & \cdots & a_{m_1m_2}B \\ \end{array} \right]$$
where the symbol $\otimes$ denotes Kronecker product, and the size of resulted $A\otimes B$ is $(m_1m_2)\times (n_1n_2)$ (i.e., $m_1\times m_2$ columns and $n_1\times n_2$ rows).

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]$ and $B=\left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10 \\ \end{array} \right]$, then, we have

$$A\otimes B=\left[ \begin{array}{cc} 1\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 2\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ 3\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] & 4\times \left[ \begin{array}{ccc} 5 & 6 & 7\\ 8 & 9 & 10\\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cccccc} 5 & 6 & 7 & 10 & 12 & 14 \\ 8 & 9 & 10 & 16 & 18 & 20 \\ 15 & 18 & 21 & 20 & 24 & 28 \\ 24 & 27 & 30 & 32 & 36 & 40 \\ \end{array} \right]\in\mathbb{R}^{4\times 6}.$$

## Khatri-Rao product (`kr_prod`)

- **Definition**:

Given two matrices $A=\left( \boldsymbol{a}_1,\boldsymbol{a}_2,...,\boldsymbol{a}_r \right)\in\mathbb{R}^{m\times r}$ and $B=\left( \boldsymbol{b}_1,\boldsymbol{b}_2,...,\boldsymbol{b}_r \right)\in\mathbb{R}^{n\times r}$ with same number of columns, then, the **Khatri-Rao product** (or **column-wise Kronecker product**) between $A$ and $B$ is given as follows,

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2,...,\boldsymbol{a}_r\otimes \boldsymbol{b}_r \right)\in\mathbb{R}^{(mn)\times r}$$
where the symbol $\odot$ denotes Khatri-Rao product, and $\otimes$ denotes Kronecker product.

- **Example**:

If $A=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]=\left( \boldsymbol{a}_1,\boldsymbol{a}_2 \right) $ and $B=\left[ \begin{array}{cc} 5 & 6 \\ 7 & 8 \\ 9 & 10 \\ \end{array} \right]=\left( \boldsymbol{b}_1,\boldsymbol{b}_2 \right) $, then, we have

$$A\odot B=\left( \boldsymbol{a}_1\otimes \boldsymbol{b}_1,\boldsymbol{a}_2\otimes \boldsymbol{b}_2 \right) $$

$$=\left[ \begin{array}{cc} \left[ \begin{array}{c} 1 \\ 3 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 5 \\ 7 \\ 9 \\ \end{array} \right] & \left[ \begin{array}{c} 2 \\ 4 \\ \end{array} \right]\otimes \left[ \begin{array}{c} 6 \\ 8 \\ 10 \\ \end{array} \right] \\ \end{array} \right]$$

$$=\left[ \begin{array}{cc} 5 & 12 \\ 7 & 16 \\ 9 & 20 \\ 15 & 24 \\ 21 & 32 \\ 27 & 40 \\ \end{array} \right]\in\mathbb{R}^{6\times 2}.$$

In [1]:
def kr_prod(a, b):
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)

In [3]:
import numpy as np
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8], [9, 10]])
print(kr_prod(A, B))

[[ 5 12]
 [ 7 16]
 [ 9 20]
 [15 24]
 [21 32]
 [27 40]]


In [4]:
def TRMF(dense_mat, sparse_mat, W, X, theta, time_lags, lambda_w, lambda_x, lambda_theta, eta, maxiter):
    dim1 = sparse_mat.shape[0]
    dim2 = sparse_mat.shape[1]
    binary_mat = np.zeros((dim1,dim2))
    position = np.where((sparse_mat > 0))
    binary_mat[position] = 1
    pos = np.where((dense_mat>0) & (sparse_mat==0))
    d = len(time_lags)
    r = theta.shape[1]

    mape = np.zeros(maxiter)
    rmse = np.zeros(maxiter)
    for iter in range(maxiter):
        var1 = X.T;
        var2 = kr_prod(var1,var1)
        var3 = np.matmul(var2,binary_mat.T)
        var4 = np.matmul(var1,sparse_mat.T)
        for i in range(dim1):
            W[i,:] = np.matmul(np.linalg.inv((var3[:,i].reshape([r,r])).T+lambda_w), var4[:,i])

        var1 = W.T
        var2 = kr_prod(var1,var1)
        var3 = np.matmul(var2, binary_mat)
        var4 = np.matmul(var1, sparse_mat)
        for t in range(dim2):
            Mt = np.zeros((r,r))
            Nt = np.zeros(r)
            if t < max(time_lags):
                Pt = np.zeros((r,r))
                Qt = np.zeros(r)
            else:
                Pt = np.eye(r)
                np.einsum('ij, ij -> j', theta, X[t -time_lags, :])
                Qt = np.einsum('ij, ij -> j', theta, X[t - time_lags, :])
            if t < dim2 - np.min(time_lags):
                if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
                    index = list(range(0, d))
                else:
                    index = list(np.where((t + time_lags >= np.max(time_lags)) & (t + time_lags < dim2)))[0]
                for k in index:
                    theta0 = theta
                    theta0[k, :] = 0
                    Mt = Mt + lambda_x * np.diag(theta[k, :]**2);
                    Nt = Nt + np.multiply(theta[k,:],(X[t+time_lags[k], :] 
                                           - np.einsum('ij, ij -> j', theta0, X[t + time_lags[k] - time_lags, :])))
                X[t,:] = np.matmul(np.linalg.inv(var3[:, t].reshape([r,r]).T + lambda_x * Pt 
                                       + lambda_x * Mt + lambda_x * eta * np.eye(r)), (var4[:, t] + Qt + Nt))
            elif t >= dim2 - np.min(time_lags) and t < dim2:
                X[t, :] = np.matmul(np.linalg.inv(var3[:, t].reshape([r, r])
                                                  + lambda_x * Pt + lambda_x * eta * np.eye(r)), (var4[:, t] + Qt))
        for k in range(d):
            var1 = X[np.max(time_lags) - time_lags[k] : dim2 - time_lags[k], :]
            var2 = np.linalg.inv(np.diag(np.einsum('ij, ij -> j', var1, var1))
                                 + (lambda_theta / lambda_x) * np.eye(r))
            var3 = np.zeros(r)

            for t in range(np.max(time_lags) - time_lags[k], dim2 - time_lags[k]):
                var3 = var3 + np.multiply(X[t, :],
                                          (X[t + time_lags[k], :] 
                                           - np.einsum('ij, ij -> j', theta, X[t + time_lags[k] - time_lags, :])
                                           +np.multiply(theta[k, :], X[t,:])))
            theta[k, :] = np.matmul(var2,var3)

        mat_hat = np.matmul(W, X.T)
        mape[iter] = np.sum(np.abs(dense_mat[pos] - mat_hat[pos]) / dense_mat[pos]) / dense_mat[pos].shape[0]
        rmse[iter] = np.sqrt(np.sum((dense_mat[pos] - mat_hat[pos])**2)/dense_mat[pos].shape[0])
        if (iter + 1) % 20 == 0:
            print("iteration = %d, MAPE = %f, RMSE = %f km/h.\n"%(iter + 1, mape[iter], rmse[iter]))


In [15]:
import scipy.io

tensor = scipy.io.loadmat('Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']
random_tensor = scipy.io.loadmat('Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

# =============================================================================
### Random missing (RM) scenario:
### ------------------------------
###   missing rate | 0.2 | 0.4 |
###   rank         |  80 |  80 |
### ------------------------------
### Set the RM scenario by:
# binary_mat = np.round(random_tensor + 0.5 - missing_rate).reshape([random_tensor.shape[0], 
#                                                                    random_tensor.shape[1] 
#                                                                    * random_tensor.shape[2]])
# =============================================================================

# =============================================================================
### Non-random missing (NM) scenario:
### ------------------------------
###   missing rate | 0.2 | 0.4 |
###   rank         |  10 |  10 |
### ------------------------------
### Set the NM scenario by:
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1,i2,:] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] 
                                    * binary_tensor.shape[2]])
# # =============================================================================

sparse_mat = np.multiply(dense_mat, binary_mat)

In [16]:
import time
start = time.time()
dim1, dim2 = sparse_mat.shape
rank = 10
lambda_w = 7
lambda_x = 7
lambda_theta = 7
eta = 0.03
time_lags = np.array([1, 2, 144])
d = time_lags.shape[0]
W = 0.1 * np.random.randn(dim1, rank)
X = 0.1 * np.random.randn(dim2, rank)
theta = 0.1 * np.random.randn(d, rank)
maxiter = 1000
TRMF(dense_mat, sparse_mat, W, X, theta, time_lags, lambda_w, lambda_x, lambda_theta, eta, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

iteration = 20, MAPE = 0.106435, RMSE = 4.585074 km/h.

iteration = 40, MAPE = 0.106243, RMSE = 4.637584 km/h.

iteration = 60, MAPE = 0.106553, RMSE = 4.709060 km/h.

iteration = 80, MAPE = 0.106735, RMSE = 4.750612 km/h.

iteration = 100, MAPE = 0.106850, RMSE = 4.779041 km/h.

iteration = 120, MAPE = 0.106931, RMSE = 4.800424 km/h.

iteration = 140, MAPE = 0.106991, RMSE = 4.817222 km/h.

iteration = 160, MAPE = 0.107036, RMSE = 4.830729 km/h.

iteration = 180, MAPE = 0.107070, RMSE = 4.841763 km/h.

iteration = 200, MAPE = 0.107097, RMSE = 4.850904 km/h.

iteration = 220, MAPE = 0.107119, RMSE = 4.858583 km/h.

iteration = 240, MAPE = 0.107137, RMSE = 4.865130 km/h.

iteration = 260, MAPE = 0.107152, RMSE = 4.870798 km/h.

iteration = 280, MAPE = 0.107165, RMSE = 4.875778 km/h.

iteration = 300, MAPE = 0.107177, RMSE = 4.880219 km/h.

iteration = 320, MAPE = 0.107188, RMSE = 4.884230 km/h.

iteration = 340, MAPE = 0.107198, RMSE = 4.887893 km/h.

iteration = 360, MAPE = 0.107207, R

**Experiment results** of missing data imputation using Bayesian probabilistic matrix factorization (TRMF):

|  scenario |`rank`|`Lambda_w`|`Lambda_x`|`Lambda_theta`|`eta`|`maxiter`|         mape |        rmse |
|:----------|-----:|---------:|---------:|-------------:|----:|--------:|-------------:|------------:|
|**20%, RM**|   10 |        5 |        5 |            5 | 0.02|    1000 |  **0.102404**| **4.327213**|
|**20%, RM**|   10 |        6 |        6 |            6 | 0.02|    1000 |  **0.102395**| **4.326849**|
|**20%, RM**|   10 |        7 |        7 |            7 | 0.01|    1000 |  **0.102387**| **4.326936**|
|**20%, RM**|   10 |        7 |        7 |            7 | 0.02|    1000 |  **0.102387**| **4.325558**|
|**20%, RM**|   10 |        4 |        7 |            7 | 0.02|    1000 |  **0.102402**| **4.327858**|
|**20%, RM**|   10 |        5 |        7 |            7 | 0.02|    1000 |  **0.102400**| **4.326938**|
|**20%, RM**|   10 |        6 |        7 |            7 | 0.02|    1000 |  **0.102400**| **4.327109**|
|**20%, RM**|   10 |        7 |        7 |            7 | 0.03|    1000 |  **0.102385**| **4.325544**|
|**20%, RM**|   10 |        7 |        7 |            7 | 0.04|    1000 |  **0.102379**| **4.326228**|
|**20%, RM**|   10 |        8 |        8 |            8 | 0.02|    1000 |  **0.102391**| **4.325751**|
|**20%, RM**|   10 |       10 |       10 |           10 | 0.02|    1000 |  **0.102389**| **4.326007**|
|**20%, RM**|   20 |        7 |        7 |            7 | 0.03|    1000 |  **0.105620**| **4.928481**|
|**20%, RM**|   40 |        7 |        7 |            7 | 0.03|    1000 |  **0.121626**| **6.615652**|
|**20%, RM**|   80 |        7 |        7 |            7 | 0.03|    1000 |  **0.212120**|**15.237144**|
|**30%, RM**|   10 |        7 |        7 |            7 | 0.03|    1000 |  **0.102960**| **4.370029**|
|**30%, RM**|   20 |        7 |        7 |            7 | 0.03|    1000 |  **0.120504**| **9.430690**|
|**30%, RM**|   40 |        7 |        7 |            7 | 0.03|    1000 |  **0.133470**| **8.424800**|
|**30%, RM**|   80 |        7 |        7 |            7 | 0.03|    1000 |  **0.259760**|**17.444252**|
|**40%, RM**|    5 |        7 |        7 |            7 | 0.03|    1000 |  **0.109189**| **4.519746**|
|**40%, RM**|   10 |        7 |        7 |            7 | 0.03|    1000 |  **0.104109**| **4.440228**|
|**40%, RM**|   20 |        7 |        7 |            7 | 0.03|    1000 |  **0.114165**| **6.134754**|
|**20%, NM**|   10 |        7 |        7 |            7 | 0.03|    1000 |  **0.103440**| **4.353568**|
|**40%, NM**|   10 |        7 |        7 |            7 | 0.03|    1000 |  **0.107368**| **4.938711**|


   > The experiment relies on the *Urban traffic speed data set in Guangzhou, China*.